<a href="https://colab.research.google.com/github/ShinyRyo/100knocks-preprocess/blob/master/simplellm_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# コード

## スクリプト1

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch

In [ ]:
ls

 aimo/                                         simplellm-answer.ipynb
'AIMOインサイトプロジェクト 作業計画書.gdoc'   test.csv
 AIMOインサイトプロジェクト計画書.gdoc         updated-code-interpretation-045c86.ipynb
 data/                                         updated-code-interpretation-21ok.ipynb


In [ ]:
# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# テスト用のプロンプト
prompt = "あなたは誰ですか？自己紹介をお願いします。"

# プロンプトをトークン化
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

# テキスト生成
output = model.generate(
    **inputs,
    max_new_tokens=100,  # 生成するトークン数の最大値
    do_sample=True,
    temperature=0.9,
    top_p=0.95
)

# 生成されたテキストをデコード
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [ ]:
generated_text

'あなたは誰ですか？自己紹介をお願いします。\nすみません。私はキタムラです。何か質問があればお聞かせください。\n\n## ライフハック\nコメントを投稿する場合は、Twitter アカウントを使用してください。'

In [ ]:
# テスト用のプロンプト
prompt = "Solve the following math problem: What is the integral of x^2?"

# プロンプトをトークン化
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

# テキスト生成
output = model.generate(
    **inputs,
    max_new_tokens=100,  # 生成するトークン数の最大値
    do_sample=True,
    temperature=0.9,
    top_p=0.95
)

# 生成されたテキストをデコード
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [ ]:
generated_text

'Solve the following math problem: What is the integral of x^2?\nIn mathematics, the integral of x^2 with respect to x is (1/3)x^3 + C, where C is the constant of integration. This result can be obtained using the power rule for integration, which states that the integral of x^n with respect to x is (1/(n+1))x^(n+1) + C, where n is a constant. In this case, n = 2, so applying the power rule gives us (1'

Solve the following math problem: What is the integral of x^2?
In mathematics, the integral of x^2 with respect to x is (1/3)x^3 + C, where C is the constant of integration. This result can be obtained using the power rule for integration, which states that the integral of x^n with respect to x is (1/(n+1))x^(n+1) + C, where n is a constant. In this case, n = 2, so applying the power rule gives us (1

##スクリプト２

## スクリプト3

### 1. モデルとトークナイザーの読み込み

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch

def load_model_and_tokenizer(model_path):
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        trust_remote_code=True,
        config=config
    )
    model = model.to(model.device)
    return model, tokenizer

### 2. プロンプトを生成する

In [ ]:
import numpy as np
from numpy.random import choice

code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

def generate_prompt(problem, past_outputs=None):
    counts = np.array([2, 3])
    draw = choice(prompt_options, 1, p=counts/counts.sum())
    if past_outputs:
        previous_output = "\n".join(past_outputs)
        prompt = f"User: {draw[0].format(problem)}\n{previous_output}"
    else:
        prompt = f"User: {draw[0].format(problem)}"
    return prompt


### 3. 解答を評価する

In [ ]:
def evaluate_answer(answer):
    try:
        return int(answer.strip())  # ダミーの評価: 数値に変換できれば有効とする
    except:
        return None

### 4. モデルを使って解答を生成する

In [ ]:
def generate_answer(model, tokenizer, prompt, max_new_tokens=2048):
    model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=1.0,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

### 5. メインのロジック

In [ ]:
import time
from collections import Counter

def solve_math_problem(problem, iterations, total_time_limit, single_attempt_time_limit):
    model_path = "deepseek-ai/deepseek-math-7b-rl"
    model, tokenizer = load_model_and_tokenizer(model_path)

    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []

    initial_prompt = generate_prompt(problem)
    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            print("Total time limit exceeded.")
            break

        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            generated_text = generate_answer(model, tokenizer, prompt)
            print(f"Attempt {attempt + 1} Output:")
            print(generated_text)

            past_outputs.append(generated_text)
            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)
                break  # 有効な解答が見つかった場合、内側のループを抜ける

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延

        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"Final Answer: {final_answer}")
            break
    else:
        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"Final Answer: {final_answer}")
        else:
            print("Could not find a valid answer within the specified attempts.")
            if past_outputs:
                valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
                if valid_answers:
                    final_answer = choice(valid_answers, 1)[0]
                    print(f"Randomly selected answer from past attempts: {final_answer}")
    return final_answer

### 6. 実行部分

In [ ]:
if __name__ == "__main__":
    problem = 'Find the units digit of \( 7^{100} \).'
    iterations = 3
    total_time_limit = 300
    single_attempt_time_limit = 30

    solve_math_problem(problem, iterations, total_time_limit, single_attempt_time_limit)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

IndexError: Replacement index 1 out of range for positional args tuple

## スクリプト3.1

In [ ]:
import numpy as np
from numpy.random import choice
import time
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import os

In [ ]:
# メモリを解放する関数
def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        allocated_memory = torch.cuda.memory_allocated(0) / (1024 ** 3)
        free_memory = total_memory - allocated_memory
        print("GPUが使用可能です。")
        print(f"使用中のGPU: {gpu_name}")
        print(f"総メモリ: {total_memory:.2f} GB")
        print(f"使用中メモリ: {allocated_memory:.2f} GB")
        print(f"空きメモリ: {free_memory:.2f} GB")
        return torch.device("cuda")
    else:
        print("GPUが使用できません。CPUを使用しますか？")
        user_input = input("CPUを使用して続行しますか？ (y/n): ")
        if user_input.lower() == 'y':
            return torch.device("cpu")
        else:
            print("処理を中止します。")
            return None

In [ ]:
import torch
from numba import cuda

def reset_cuda():
    torch.cuda.empty_cache()
    cuda.select_device(0)
    cuda.close()

reset_cuda()


In [ ]:
clear_memory()
reset_cuda()

In [ ]:
check_gpu()

GPUが使用可能です。
使用中のGPU: Tesla T4
総メモリ: 14.75 GB
使用中メモリ: 0.01 GB
空きメモリ: 14.74 GB


device(type='cuda')

In [ ]:
!nvidia-smi


Sun Jun 30 10:29:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0              36W /  70W |      3MiB / 15360MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 1. モデルとトークナイザーの読み込み

In [ ]:
# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path):
    print("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        trust_remote_code=True,
        config=config
    )
    print("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

### 2. プロンプトを生成する

In [ ]:
# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

In [ ]:
code_template = """以下の数学の問題を解いてください（正の数値で回答）:
"{}"
まず、問題を解決するための手順を列挙し、それぞれの手順で呼び出す関数を示してください。全てのステップをコメントで丁寧に記述し、最終的な答えを \\boxed{} 内に出力してください。

アプローチ:"""

cot_template = """以下の数学の問題を解いてください（正の数値で回答）:
"{}"
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \\boxed{} 内に出力してください。

"""

prompt_options = [code_template, cot_template]

In [ ]:
# プロンプトを生成する関数
def generate_prompt(problem, past_outputs=None):
    counts = np.array([2, 3])
    draw = choice(prompt_options, 1, p=counts/counts.sum())
    if past_outputs:
        previous_output = "\n".join(past_outputs)
        prompt = f"User: {draw[0].format(problem, {})}\n{previous_output}"
    else:
        prompt = f"User: {draw[0].format(problem, {})}"
    return prompt

In [ ]:
# モデルを使って解答を生成する関数
def generate_answer(model, tokenizer, prompt, device, max_new_tokens=2048):
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=1.0,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

### 3. 解答を評価する

In [ ]:
# 解答を評価する関数
def evaluate_answer(answer):
    match = re.search(r'\\boxed\{(\d+)\}', answer)
    if match:
        print(f"Extracted answer: {match.group(1)}")  # デバッグメッセージ
        return int(match.group(1))
    else:
        print("No valid answer found in:", answer)  # デバッグメッセージ
        return None

In [ ]:
# 検証過程と答えを保存する関数
def save_results(past_outputs, final_answer, filepath="results.json"):
    results = {
        "past_outputs": past_outputs,
        "final_answer": final_answer
    }
    with open(filepath, "w") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

### 4. モデルを使って解答を生成する

In [ ]:
# 問題を解決するメインロジック
def solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device):
    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []

    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            print("合計時間制限を超えました。")
            break

        print(f"試行 {attempt + 1} 回目:")
        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            generated_text = generate_answer(model, tokenizer, prompt, device)
            print(f"試行 {attempt + 1} 回目の回答:")
            print(generated_text)

            past_outputs.append(generated_text)
            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)
                break  # 有効な解答が見つかった場合、内側のループを抜ける

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延
        # 各試行の終了後にメモリを解放
        clear_memory()

        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"最終解答: {final_answer}")
            break
    else:
        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"最終解答: {final_answer}")
        else:
            print("指定された試行回数内で有効な解答が見つかりませんでした。")
            if past_outputs:
                valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
                if valid_answers:
                    final_answer = choice(valid_answers, 1)[0]
                    print(f"過去の試行からランダムに選ばれた解答: {final_answer}")
    # 全試行の終了後にメモリを解放
    clear_memory()
    # 全試行の終了後にメモリを解放
    clear_memory()

    # 検証過程と答えを保存
    save_results(past_outputs, final_answer)

    return final_answer

In [ ]:
# 問題を解決するメインロジック
def solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device):
    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []

    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            print("合計時間制限を超えました。")
            break

        print(f"試行 {attempt + 1} 回目:")
        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            generated_text = generate_answer(model, tokenizer, prompt, device)
            print(f"思考 {attempt + 1} 回目の回答:")
            print(generated_text)

            past_outputs.append(generated_text)
            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)
                # breakを削除して、指定回数すべての試行を行う

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延

    if answers:
        final_answer = Counter(answers).most_common(1)[0][0]
        print(f"最終解答: {final_answer}")
    else:
        print("指定された試行回数内で有効な解答が見つかりませんでした。")
        if past_outputs:
            valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
            if valid_answers:
                final_answer = choice(valid_answers, 1)[0]
                print(f"過去の試行からランダムに選ばれた解答: {final_answer}")
    return final_answer


### 5. メインのロジック

In [ ]:
# メイン関数
def main():
    # 問題の設定
    problem = 'Find the units digit of \( 7^{100} \).'
    iterations = 3
    total_time_limit = 300  # 秒
    single_attempt_time_limit = 30  # 秒

    # GPUの確認
    device = check_gpu()
    if device is None:
        print("処理を中止します。")
        return

    # モデルとトークナイザーの読み込み
    model_path = "deepseek-ai/deepseek-math-7b-rl"
    model, tokenizer = load_model_and_tokenizer(model_path)
    model = model.to(device)
    clear_memory()  # モデルのロード後にメモリを解放
    print("モデルのロード後にメモリを解放しました")

    # 初期プロンプトの生成
    initial_prompt = generate_prompt(problem)

    # 問題を解決するためのメインロジック
    final_answer = solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device)
    clear_memory()
    del model
    print("メモリを解放しました")
    # # 結果の表示
    # if final_answer is not None:
    #     print(f"最終解答: {final_answer}")
    # else:
    #     print("指定された試行回数内で有効な解答が見つかりませんでした。")

### 6. 実行部分

In [ ]:
# import torch
# import gc
# # GPUメモリキャッシュをクリア
# torch.cuda.empty_cache()
# # Pythonのガベージコレクタを呼び出し
# gc.collect()

In [ ]:
import torch
import gc
import re
import json
if __name__ == "__main__":
    main()

GPUが使用可能です。
使用中のGPU: Tesla T4
総メモリ: 14.75 GB
使用中メモリ: 0.01 GB
空きメモリ: 14.74 GB
モデルとトークナイザーを読み込んでいます...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

モデルとトークナイザーの読み込みが完了しました。


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


...

試行 2 回目:
思考 2 回目の回答:
User: 以下の数学の問題を解いてください（正の数値で回答）:
"Find the units digit of \( 7^{100} \)."
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \boxed{} 内に出力してください。

## 問題分解
この問題は、単一の数値を見つけるだけです。その数は、$7^{100}$ の末尾の数字です。単にこの数を計算するのは大変なので、より小さな数値で試してみて、パターンを見つけることができれば、より大きな数値にも応用できるかもしれません。

## 小さな指数で試す
小さな指数で試してみましょう。
$7^1 = 7$ の末尾の数字は $7$ です。
$7^2 = 49$ の末尾の数字は $9$ です。
$7^3 = 343$ の末尾の数字は $3$ です。
$7^4 = 2401$ の末尾の数字は $1$ です。
$7^5 = 16807$ の末尾の数字は $7$ です。

ここで、$7^1$, $7^5$ の末尾の数字は同じです。およそ $7^n$ の末尾の数字は $n$ が $4$ の倍数になると同じ数字になることがわかります。

## 大きな指数で試す
では、$n = 100$ は $4$ の倍数です。実際に計算してみましょう。

$7^{100}$ の末尾の数字を見つけるのは大変なので、$7^4$ の末尾の数字に注目してみましょう。

$7^1 = 7$
$7^2 = 49$
$7^3 = 343$
$7^4 = 2401$ の末尾の数字は $1$ です。

$7^{100}$ は $7^4$ の $25$ 個の積なので、末尾の数字は $7^4$ の末尾の数字と同じです。したがって、末尾の数字は $1$ です。

## 解決策
よって、$7^{100}$ の末尾の数字は $1$ です。

## 解決答え
したがって、$7^{100}$ の末尾の数字は $\boxed{1}$ です。
Extracted answer: 1


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


試行 3 回目:
思考 3 回目の回答:
User: 以下の数学の問題を解いてください（正の数値で回答）:
"Find the units digit of \( 7^{100} \)."
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \boxed{} 内に出力してください。

## 問題分解
この問題は、7の累乗の末尾の数字（単位数）を求めることです。7の累乗を計算すると、次のようになります：
\[7^1 = 7, 7^2 = 49, 7^3 = 343, 7^4 = 2401, \dotsb.\]
これらの数の単位数は7, 9, 3, 1, ...と周期的に繰り返します。周期は4です。

## 解決策
100 ÷ 4 = 25 ですので、7の100乗の単位数は、7の4乗の単位数と同じです。したがって、7の100乗の単位数は1です。

## 答え
よって、7の100乗の単位数は1です。そのため、求める答えは $1$ です。

## 解決答案
よって、求める答えは $1$ です。
最終的な答えは $\boxed{1}$ です。
Extracted answer: 1


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


思考 3 回目の回答:
User: 以下の数学の問題を解いてください（正の数値で回答）:
"Find the units digit of \( 7^{100} \)."
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \boxed{} 内に出力してください。

## 問題分解
この問題を解くためには、まず、$7$ の累乗の単位数位を調べてみる必要がある。そのためには、$7$ の累乗を計算してみるとよい。

## 単位数位の規則性を調べる
$7^1 = 7$,
$7^2 = 49$,
$7^3 = 343$,
$7^4 = 2401$,
$7^5 = 16807$,
$7^6 = 117649$,
...
というように、単位数位は $7, 9, 3, 1$ と $4$ つごとに繰り返していることが分かる。

## 問題に戻って単位数位を求める
$100$ は $4$ の倍数なので、$7^{100}$ の単位数位は $1$ である。

## 解決
よって、$7^{100}$ の単位数位は $1$。

## 最終解答
$7^{100}$ の単位数位は $1$ である。よって、求める単位数位は $1$。

それでは、答えを $\boxed{1}$ として出力します。
Extracted answer: 1
最終解答: 1
メモリを解放しました


## スクリプト4

In [ ]:
import torch
import gc

# 使用しているテンソルを削除
def clear_all_tensors():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                del obj
        except:
            pass

clear_all_tensors()
torch.cuda.empty_cache()
gc.collect()


108

In [ ]:
globals()[_8]

KeyError: 0

In [ ]:
for name in dir():
    print(name)

AutoConfig
AutoModelForCausalLM
AutoTokenizer
Counter
In
Out
_
_10
_12
_13
_14
_17
_19
_7
_8
_9
__
___
__builtin__
__builtins__
__doc__
__loader__
__name__
__package__
__spec__
_dh
_i
_i1
_i10
_i11
_i12
_i13
_i14
_i15
_i16
_i17
_i18
_i19
_i2
_i20
_i21
_i3
_i4
_i5
_i6
_i7
_i8
_i9
_ih
_ii
_iii
_oh
check_gpu
choice
clear_all_tensors
clear_gpu_memory
clear_memory
device
evaluate_answer
exit
gc
generate_answer
generate_prompt
get_ipython
load_model_and_tokenizer
main
np
obj
os
print_gpu_memory_status
print_memory_status
quit
reset_cuda
reset_device
save_results
solve_math_problem
time
torch


In [ ]:
import torch

# 現在のデバイス（例：GPU 0）
device = torch.device("cuda:0")

# 現在のデバイスに存在するすべてのテンソルを取得
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) and obj.device == device:
            print(f"Tensor: {obj}, Size: {obj.size()}, Device: {obj.device}, Type: {obj.dtype}")
    except ReferenceError:
        pass


Tensor: Parameter containing:
tensor([[ 0.0898,  0.0635, -0.0051,  ...,  0.3496,  0.0309,  0.0781],
        [ 0.1357,  0.3730, -0.0889,  ...,  0.0933,  0.0452,  0.4277],
        [ 0.0645, -0.0854, -0.2734,  ..., -0.1348,  0.0500,  0.0967],
        ...,
        [-0.1040,  0.1865, -0.0967,  ..., -0.0693, -0.0571,  0.1494],
        [-0.1021,  0.1895, -0.1060,  ..., -0.0664, -0.0649,  0.1465],
        [-0.0933,  0.1797, -0.1108,  ..., -0.0635, -0.0532,  0.1445]],
       device='cuda:0', dtype=torch.bfloat16, requires_grad=True), Size: torch.Size([102400, 4096]), Device: cuda:0, Type: torch.bfloat16
Tensor: Parameter containing:
tensor([[ 3.0029e-02,  1.1035e-01,  5.8203e-01,  ..., -3.1250e-01,
          1.3770e-01, -5.8289e-03],
        [-9.0332e-02, -6.5430e-02,  6.8750e-01,  ...,  3.0518e-02,
         -8.8867e-02, -9.7656e-02],
        [ 3.0060e-03,  3.7354e-02,  2.9102e-01,  ...,  1.3867e-01,
          8.5449e-02, -2.0605e-01],
        ...,
        [ 4.7607e-03,  8.9111e-03,  8.7280e-03

In [ ]:
import torch

# 現在アクティブなテンソルを取得する
active_tensors = [obj for obj in gc.get_objects() if torch.is_tensor(obj)]

# アクティブなテンソルの情報を表示
for tensor in active_tensors:
    print(tensor.size(), tensor.device, tensor.dtype)


/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


ReferenceError: weakly-referenced object no longer exists

In [ ]:
torch.cuda.get_device_properties(0).total_memory/ (1024 ** 3)

14.74810791015625

In [ ]:
torch.cuda.memory_allocated(0) / (1024 ** 3)

12.871573448181152

In [ ]:
torch.cuda.memory_reserved(0) / (1024 ** 3)

12.873046875

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
import numpy as np
from numpy.random import choice
import time
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import os
import gc

# メモリを解放する関数
def clear_memory():
    if torch.cuda.is_available():
        try:
            torch.cuda.empty_cache()
        except RuntimeError as e:
            print(f"CUDA error during empty_cache: {e}")
    gc.collect()

# CUDAコンテキストのリセット
def reset_cuda():
    if torch.cuda.is_available():
        try:
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        except RuntimeError as e:
            print(f"CUDA error during reset: {e}")
    gc.collect()

# デバイスのリセット
def reset_device():
    if torch.cuda.is_available():
        try:
            torch.cuda.device_reset()
        except RuntimeError as e:
            print(f"CUDA error during device reset: {e}")

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        allocated_memory = torch.cuda.memory_allocated(0) / (1024 ** 3)
        free_memory = total_memory - allocated_memory
        print("GPUが使用可能です。")
        print(f"使用中のGPU: {gpu_name}")
        print(f"総メモリ: {total_memory:.2f} GB")
        print(f"使用中メモリ: {allocated_memory:.2f} GB")
        print(f"空きメモリ: {free_memory:.2f} GB")
        return torch.device("cuda")
    else:
        print("GPUが使用できません。CPUを使用しますか？")
        user_input = input("CPUを使用して続行しますか？ (y/n): ")
        if user_input.lower() == 'y':
            return torch.device("cpu")
        else:
            print("処理を中止します。")
            return None

# メモリの状況を確認する関数
def print_memory_status():
    print(f"CUDAメモリ使用状況:")
    os.system("nvidia-smi")

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path, device):
    print("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        trust_remote_code=True,
        config=config
    )
    model.to(device)
    print("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

# プロンプトを生成する関数
def generate_prompt(problem, past_outputs=None):
    counts = np.array([2, 3])
    draw = choice(prompt_options, 1, p=counts/counts.sum())
    if past_outputs:
        previous_output = "\n".join(past_outputs)
        prompt = f"User: {draw[0].format(problem, {})}\n{previous_output}"
    else:
        prompt = f"User: {draw[0].format(problem, {})}"
    return prompt

# モデルを使って解答を生成する関数
def generate_answer(model, tokenizer, prompt, device, max_new_tokens=2048):
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=1.0,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

# 解答を評価する関数
def evaluate_answer(answer):
    match = re.search(r'\\boxed\{(\d+)\}', answer)
    if match:
        print(f"Extracted answer: {match.group(1)}")  # デバッグメッセージ
        return int(match.group(1))
    else:
        print("No valid answer found in:", answer)  # デバッグメッセージ
        return None

# 検証過程と答えを保存する関数
def save_results(past_outputs, final_answer, filepath="results.json"):
    results = {
        "past_outputs": past_outputs,
        "final_answer": final_answer
    }
    with open(filepath, "w") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

# 問題を解決するメインロジック
def solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device):
    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []

    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            print("合計時間制限を超えました。")
            break

        print(f"試行 {attempt + 1} 回目:")
        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            generated_text = generate_answer(model, tokenizer, prompt, device)
            print(f"試行 {attempt + 1} 回目の回答:")
            print(generated_text)

            past_outputs.append(generated_text)
            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延
        clear_memory()  # 各試行の終了後にメモリを解放

    if answers:
        final_answer = Counter(answers).most_common(1)[0][0]
        print(f"最終解答: {final_answer}")
    else:
        print("指定された試行回数内で有効な解答が見つかりませんでした。")
        if past_outputs:
            valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
            if valid_answers:
                final_answer = choice(valid_answers, 1)[0]
                print(f"過去の試行からランダムに選ばれた解答: {final_answer}")

    clear_memory()  # 全試行の終了後にメモリを解放
    save_results(past_outputs, final_answer)
    return final_answer

# メイン関数
def main():
    # 環境変数の設定
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    os.environ['CUDA_CACHE_MAXSIZE'] = '0'

    # 問題の設定
    problem = 'Find the units digit of \( 7^{100} \).'
    iterations = 3
    total_time_limit = 300  # 秒
    single_attempt_time_limit = 30  # 秒

    # GPUの確認
    device = check_gpu()
    if device is None:
        print("処理を中止します。")
        return

    # メモリ状況の表示
    print_memory_status()

    # メモリを解放
    clear_memory()
    reset_cuda()

    # メモリ状況の再表示
    print_memory_status()

    # モデルとトークナイザーの読み込み
    model_path = "deepseek-ai/deepseek-math-7b-rl"
    model, tokenizer = load_model_and_tokenizer(model_path, device)

    # 初期プロンプトの生成
    initial_prompt = generate_prompt(problem)

    # 問題を解決するためのメインロジック
    final_answer = solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device)

    # 最終メモリ状況の表示
    print_memory_status()

    # メモリを再度解放
    clear_memory()
    reset_cuda()

if __name__ == "__main__":
    main()

GPUが使用可能です。
使用中のGPU: Tesla T4
総メモリ: 14.75 GB
使用中メモリ: 14.62 GB
空きメモリ: 0.13 GB
CUDAメモリ使用状況:
CUDAメモリ使用状況:
モデルとトークナイザーを読み込んでいます...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 

In [ ]:
# 最終メモリ状況の表示
print_memory_status()

# メモリを再度解放
clear_memory()
reset_cuda()

CUDAメモリ使用状況:


In [ ]:
if __name__ == "__main__":
    main()

NameError: name 'main' is not defined

In [ ]:
# すべての変数を削除
for name in dir():
    if not name.startswith('_'):
        del globals()[name]


In [ ]:
print("\nGPUメモリを解放します...")
del model
clear_gpu_memory()

# メモリ解放後のGPUメモリの使用状況を表示
print("メモリ解放後のGPUメモリ使用状況:")
print_gpu_memory_status()


GPUメモリを解放します...


NameError: name 'model' is not defined

In [ ]:
import torch
import gc

# GPUメモリの使用状況を表示する関数
def print_gpu_memory_status():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        allocated_memory = torch.cuda.memory_allocated(0) / (1024 ** 3)
        reserved_memory = torch.cuda.memory_reserved(0) / (1024 ** 3)
        free_memory = total_memory - allocated_memory
        print(f"総メモリ: {total_memory:.2f} GB")
        print(f"使用中メモリ: {allocated_memory:.2f} GB")
        print(f"予約済みメモリ: {reserved_memory:.2f} GB")
        print(f"空きメモリ: {free_memory:.2f} GB")
    else:
        print("GPUが使用できません。")

# GPUメモリを解放する関数
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("GPUメモリを解放しました。")

# メイン関数
def main():
    # GPUメモリの使用状況を表示
    print("初期状態のGPUメモリ使用状況:")
    print_gpu_memory_status()

    # ダミーデータをGPUにロードしてみる
    print("\nダミーデータをGPUにロードします...")
    tensor = torch.randn((10000, 10000), device="cuda")

    # ダミーデータをロードした後のGPUメモリの使用状況を表示
    print("ダミーデータをロードした後のGPUメモリ使用状況:")
    print_gpu_memory_status()

    # GPUメモリを解放
    print("\nGPUメモリを解放します...")
    del tensor
    clear_gpu_memory()

    # メモリ解放後のGPUメモリの使用状況を表示
    print("メモリ解放後のGPUメモリ使用状況:")
    print_gpu_memory_status()

if __name__ == "__main__":
    main()

初期状態のGPUメモリ使用状況:
総メモリ: 14.75 GB
使用中メモリ: 14.62 GB
予約済みメモリ: 14.62 GB
空きメモリ: 0.13 GB

ダミーデータをGPUにロードします...


OutOfMemoryError: CUDA out of memory. Tried to allocate 382.00 MiB. GPU 

## スクリプト５

In [ ]:
import torch
import gc

# GPUメモリの使用状況を詳細に表示する関数
def print_gpu_memory_status():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        allocated_memory = torch.cuda.memory_allocated(0) / (1024 ** 3)
        reserved_memory = torch.cuda.memory_reserved(0) / (1024 ** 3)
        free_memory = total_memory - allocated_memory
        print(f"総メモリ: {total_memory:.2f} GB")
        print(f"使用中メモリ (torch.cuda.memory_allocated(0)): {allocated_memory:.2f} GB ({torch.cuda.memory_allocated(0)} バイト)")
        print(f"予約済みメモリ: {reserved_memory:.2f} GB")
        print(f"空きメモリ: {free_memory:.2f} GB")
    else:
        print("GPUが使用できません。")

# GPUメモリを解放する関数
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("GPUメモリを解放しました。")

# メイン関数
def main():
    # GPUメモリの使用状況を表示
    print("初期状態のGPUメモリ使用状況:")
    print_gpu_memory_status()

    # ダミーデータをGPUにロードしてみる
    print("\nダミーデータをGPUにロードします...")
    tensor = torch.randn((100000, 10000), device="cuda")

    # ダミーデータをロードした後のGPUメモリの使用状況を表示
    print("ダミーデータをロードした後のGPUメモリ使用状況:")
    print_gpu_memory_status()

    # アクティブなテンソルを削除
    print("\nアクティブなテンソルを削除します...")
    del tensor

    # GPUメモリを解放
    print("GPUメモリを解放します...")
    clear_gpu_memory()

    # メモリ解放後のGPUメモリの使用状況を表示
    print("メモリ解放後のGPUメモリ使用状況:")
    print_gpu_memory_status()

if __name__ == "__main__":
    main()

初期状態のGPUメモリ使用状況:
総メモリ: 14.75 GB
使用中メモリ (torch.cuda.memory_allocated(0)): 0.00 GB (0 バイト)
予約済みメモリ: 0.00 GB
空きメモリ: 14.75 GB

ダミーデータをGPUにロードします...
ダミーデータをロードした後のGPUメモリ使用状況:
総メモリ: 14.75 GB
使用中メモリ (torch.cuda.memory_allocated(0)): 3.73 GB (4000000000 バイト)
予約済みメモリ: 3.73 GB
空きメモリ: 11.02 GB

アクティブなテンソルを削除します...
GPUメモリを解放します...
GPUメモリを解放しました。
メモリ解放後のGPUメモリ使用状況:
総メモリ: 14.75 GB
使用中メモリ (torch.cuda.memory_allocated(0)): 0.00 GB (0 バイト)
予約済みメモリ: 0.00 GB
空きメモリ: 14.75 GB


In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import os
import gc

def check_memory():
    gpu_memory, gpu_reserved, gpu_allocated, gpu_free = 0, 0, 0, 0
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated

    total_memory = int(os.popen('free -m').readlines()[1].split()[1]) / 1024
    used_memory = int(os.popen('free -m').readlines()[1].split()[2]) / 1024
    free_memory = total_memory - used_memory

    return {
        "gpu_memory": gpu_memory,
        "gpu_reserved": gpu_reserved,
        "gpu_allocated": gpu_allocated,
        "gpu_free": gpu_free,
        "total_memory": total_memory,
        "used_memory": used_memory,
        "free_memory": free_memory,
    }

def display_memory_info():
    memory_info = check_memory()
    print(f"GPU メモリ: {memory_info['gpu_memory']} GB")
    print(f"GPU 予約済み: {memory_info['gpu_reserved']} GB")
    print(f"GPU 使用中: {memory_info['gpu_allocated']} GB")
    print(f"GPU 空き: {memory_info['gpu_free']} GB")
    print(f"CPU メモリ: {memory_info['total_memory']} GB")
    print(f"CPU 使用中: {memory_info['used_memory']} GB")
    print(f"CPU 空き: {memory_info['free_memory']} GB")

def load_model_and_tokenizer(model_path, device):
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        trust_remote_code=True,
        config=config
    )
    model.to(device)  # モデルをGPUに移行
    return model, tokenizer

def unload_model(model):
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

In [ ]:
del model

In [ ]:
# メイン部分
if __name__ == "__main__":
    model_path = "deepseek-ai/deepseek-math-7b-rl"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # モデルの読み込み前にメモリ使用状況を表示
    print("モデル読み込み前のメモリ使用状況:")
    display_memory_info()

    # モデルを読み込む
    model, tokenizer = load_model_and_tokenizer(model_path, device)

    # モデルの読み込み後にメモリ使用状況を表示
    print("モデル読み込み後のメモリ使用状況:")
    display_memory_info()

    # モデルの削除
    del model
    #unload_model(model)

    # モデル削除後のメモリ使用状況を表示
    print("モデル削除後のメモリ使用状況:")
    display_memory_info()

モデル読み込み前のメモリ使用状況:
GPU メモリ: 14.74810791015625 GB
GPU 予約済み: 12.873046875 GB
GPU 使用中: 0.0 GB
GPU 空き: 14.74810791015625 GB
CPU メモリ: 50.9931640625 GB
CPU 使用中: 1.666015625 GB
CPU 空き: 49.3271484375 GB


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

モデル読み込み後のメモリ使用状況:
GPU メモリ: 14.74810791015625 GB
GPU 予約済み: 12.873046875 GB
GPU 使用中: 12.871573448181152 GB
GPU 空き: 1.8765344619750977 GB
CPU メモリ: 50.9931640625 GB
CPU 使用中: 1.6728515625 GB
CPU 空き: 49.3203125 GB
モデル削除後のメモリ使用状況:
GPU メモリ: 14.74810791015625 GB
GPU 予約済み: 12.873046875 GB
GPU 使用中: 0.0 GB
GPU 空き: 14.74810791015625 GB
CPU メモリ: 50.9931640625 GB
CPU 使用中: 1.6728515625 GB
CPU 空き: 49.3203125 GB


In [ ]:
torch.cuda.memory_allocated(0)

0

## スクリプト6

In [ ]:
import numpy as np
from numpy.random import choice
import time
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import os

In [ ]:
# メモリを解放する関数
def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        allocated_memory = torch.cuda.memory_allocated(0) / (1024 ** 3)
        free_memory = total_memory - allocated_memory
        print("GPUが使用可能です。")
        print(f"使用中のGPU: {gpu_name}")
        print(f"総メモリ: {total_memory:.2f} GB")
        print(f"使用中メモリ: {allocated_memory:.2f} GB")
        print(f"空きメモリ: {free_memory:.2f} GB")
        return torch.device("cuda")
    else:
        print("GPUが使用できません。CPUを使用しますか？")
        user_input = input("CPUを使用して続行しますか？ (y/n): ")
        if user_input.lower() == 'y':
            return torch.device("cpu")
        else:
            print("処理を中止します。")
            return None

In [ ]:
import torch
from numba import cuda

def reset_cuda():
    torch.cuda.empty_cache()
    cuda.select_device(0)
    cuda.close()

reset_cuda()


### 1. モデルとトークナイザーの読み込み

In [ ]:
# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path):
    print("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        trust_remote_code=True,
        config=config
    )
    print("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

### 2. プロンプトを生成する

In [ ]:
# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

In [ ]:
code_template = """以下の数学の問題を解いてください（正の数値で回答）:
"{}"
まず、問題を解決するための手順を列挙し、それぞれの手順で呼び出す関数を示してください。全てのステップをコメントで丁寧に記述し、最終的な答えを \\boxed{} 内に出力してください。

アプローチ:"""

cot_template = """以下の数学の問題を解いてください（正の数値で回答）:
"{}"
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \\boxed{} 内に出力してください。

"""

prompt_options = [code_template, cot_template]

In [ ]:
# プロンプトを生成する関数
def generate_prompt(problem, past_outputs=None):
    counts = np.array([2, 3])
    draw = choice(prompt_options, 1, p=counts/counts.sum())
    if past_outputs:
        previous_output = "\n".join(past_outputs)
        prompt = f"User: {draw[0].format(problem, {})}\n{previous_output}"
    else:
        prompt = f"User: {draw[0].format(problem, {})}"
    return prompt

In [ ]:
# モデルを使って解答を生成する関数
def generate_answer(model, tokenizer, prompt, device, max_new_tokens=2048):
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=1.0,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

### 3. 解答を評価する

In [ ]:
# 解答を評価する関数
def evaluate_answer(answer):
    match = re.search(r'\\boxed\{(\d+)\}', answer)
    if match:
        print(f"Extracted answer: {match.group(1)}")  # デバッグメッセージ
        return int(match.group(1))
    else:
        print("No valid answer found in:", answer)  # デバッグメッセージ
        return None

In [ ]:
# 検証過程と答えを保存する関数
def save_results(past_outputs, final_answer, filepath="results.json"):
    results = {
        "past_outputs": past_outputs,
        "final_answer": final_answer
    }
    with open(filepath, "w") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

### 4. モデルを使って解答を生成する

In [ ]:
# 問題を解決するメインロジック
def solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device):
    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []

    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            print("合計時間制限を超えました。")
            break

        print(f"試行 {attempt + 1} 回目:")
        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            generated_text = generate_answer(model, tokenizer, prompt, device)
            print(f"試行 {attempt + 1} 回目の回答:")
            print(generated_text)

            past_outputs.append(generated_text)
            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)
                break  # 有効な解答が見つかった場合、内側のループを抜ける

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延
        # 各試行の終了後にメモリを解放
        clear_memory()

        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"最終解答: {final_answer}")
            break
    else:
        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"最終解答: {final_answer}")
        else:
            print("指定された試行回数内で有効な解答が見つかりませんでした。")
            if past_outputs:
                valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
                if valid_answers:
                    final_answer = choice(valid_answers, 1)[0]
                    print(f"過去の試行からランダムに選ばれた解答: {final_answer}")
    # 全試行の終了後にメモリを解放
    clear_memory()
    # 全試行の終了後にメモリを解放
    clear_memory()

    # 検証過程と答えを保存
    save_results(past_outputs, final_answer)

    return final_answer

In [ ]:
# 問題を解決するメインロジック
def solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device):
    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []

    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            print("合計時間制限を超えました。")
            break

        print(f"試行 {attempt + 1} 回目:")
        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            generated_text = generate_answer(model, tokenizer, prompt, device)
            print(f"思考 {attempt + 1} 回目の回答:")
            print(generated_text)

            past_outputs.append(generated_text)
            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)
                # breakを削除して、指定回数すべての試行を行う

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延

    if answers:
        final_answer = Counter(answers).most_common(1)[0][0]
        print(f"最終解答: {final_answer}")
    else:
        print("指定された試行回数内で有効な解答が見つかりませんでした。")
        if past_outputs:
            valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
            if valid_answers:
                final_answer = choice(valid_answers, 1)[0]
                print(f"過去の試行からランダムに選ばれた解答: {final_answer}")
    return final_answer


### 5. メインのロジック

In [ ]:
# メイン関数
def main():
    # 問題の設定
    problem = 'Find the units digit of \( 7^{100} \).'
    iterations = 3
    total_time_limit = 300  # 秒
    single_attempt_time_limit = 30  # 秒

    # GPUの確認
    device = check_gpu()
    if device is None:
        print("処理を中止します。")
        return

    # モデルとトークナイザーの読み込み
    model_path = "deepseek-ai/deepseek-math-7b-rl"
    model, tokenizer = load_model_and_tokenizer(model_path)
    model = model.to(device)
    clear_memory()  # モデルのロード後にメモリを解放
    print("モデルのロード後にメモリを解放しました")

    # 初期プロンプトの生成
    initial_prompt = generate_prompt(problem)

    # 問題を解決するためのメインロジック
    final_answer = solve_math_problem(problem, model, tokenizer, initial_prompt, iterations, total_time_limit, single_attempt_time_limit, device)
    clear_memory()
    del model
    print("メモリを解放しました")
    # # 結果の表示
    # if final_answer is not None:
    #     print(f"最終解答: {final_answer}")
    # else:
    #     print("指定された試行回数内で有効な解答が見つかりませんでした。")

### 6. 実行部分

In [ ]:
import torch
import gc
import re
import json
if __name__ == "__main__":
    main()

GPUが使用可能です。
使用中のGPU: Tesla T4
総メモリ: 14.75 GB
使用中メモリ: 0.00 GB
空きメモリ: 14.75 GB
モデルとトークナイザーを読み込んでいます...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

モデルとトークナイザーの読み込みが完了しました。
モデルのロード後にメモリを解放しました
試行 1 回目:


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


思考 1 回目の回答:
User: 以下の数学の問題を解いてください（正の数値で回答）:
"Find the units digit of \( 7^{100} \)."
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \boxed{} 内に出力してください。

## 問題分解
この問題は、7を100乗した時の一の位を求める問題です。7を何乗しても一の位は7になりません。一の位には周期が存在します。そこで、7の累乗の一の位を一周期ごとに調べ、どのような周期になるかを探る方法が有効です。

## 解決方針
7を累乗して一の位を調べます。
\[
\begin{aligned} 7^1 &= 7, \\ 7^2 &= 49, \\ 7^3 &= 343, \\ 7^4 &= 2401, \\ 7^5 &= 16807, \\ 7^6 &= 117649, \\ 7^7 &= 823543, \\ 7^8 &= 5764801, \\ 7^9 &= 40353607, \\ 7^{10} &= 282475249. \end{aligned}
\]
一の位を見ると、7, 9, 3, 1の順に繰り返しています。このように、7の累乗の一の位は4つの数字を繰り返します。つまり、一の位の周期は4です。

100は4の倍数です。なので、100乗すると一の位の数字は周期の最後の数字になります。

この周期の最後の数字は1です。

## 解決答案
よって、\( 7^{100} \) の一の位は1です。答えは $1$。

## 解決答案
よって、$7^{100}$の一の位の数字は1です。
最終的な答えは $1$ です。$\boxed{1}$
Extracted answer: 1


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


試行 2 回目:
思考 2 回目の回答:
User: 以下の数学の問題を解いてください（正の数値で回答）:
"Find the units digit of \( 7^{100} \)."
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \boxed{} 内に出力してください。

## 問題分解
単位数（末尾の数字）を見つけるためには、まず $7^1, 7^2, 7^3, ..., 7^n$ ($n$ は任意の整数）の単位数を観察すると良い。そうすれば、どの時点で単位数が循環するかを発見できるはずである。

## 解決策
$7^1 = 7$
$7^2 = 49$
$7^3 = 343$
$7^4 = 2401$
$7^5 = 16807$
...
$7^6 = 117649$
$7^7 = 823543$
$7^8 = 5764801$
$7^9 = 40353607$
$7^{10} = 282475249$
...
お気づきかもしれないが、$7^1, 7^2, 7^3, ..., 7^6$ の単位数はそれぞれ $7, 9, 3, 1, 7, 9$ であり、これが循環する。つまり、$7^n$ の単位数は $n$ を $6$ で割った余りによって決まる。

したがって、$7^{100}$ の単位数は $100$ を $6$ で割った余りと同じである。$100$ を $6$ で割ると商は $16$ で余りは $4$ である。

よって、$7^{100}$ の単位数は $7^4$ の単位数と同じであり、$1$ である。

## 解決答案
よって、$7^{100}$ の単位数は $1$ である。

正の数値で回答すると、求める単位数は $1$ である。

最終解答： $\boxed{1}$
Extracted answer: 1


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


試行 3 回目:
思考 3 回目の回答:
User: 以下の数学の問題を解いてください（正の数値で回答）:
"Find the units digit of \( 7^{100} \)."
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \boxed{} 内に出力してください。

## 問題分解
この問題は、単純に $7^{100}$ の末桁を求めるだけです。しかし、指数が大きいので単純計算で求めるのは難しいです。そこで、$7^n$ の末桁の規則性を調べましょう。

## 末桁の規則性を調べる
$7^1 = 7$, $7^2 = 49$, $7^3 = 343$, $7^4 = 2401$, $7^5 = 16807$, $7^6 = 117649$, ...

このように計算すると、$7^n$ の末桁は以下のような規則性があることがわかります。
$$
\begin{align*}
7^1 &\quad \text{の末桁は} \quad 7, \\
7^2 &\quad \text{の末桁は} \quad 9, \\
7^3 &\quad \text{の末桁は} \quad 3, \\
7^4 &\quad \text{の末桁は} \quad 1, \\
7^5 &\quad \text{の末桁は} \quad 7, \\
7^6 &\quad \text{の末桁は} \quad 9, \\
&\quad \vdots
\end{align*}
$$
このように、$7^n$ の末桁は $7, 9, 3, 1$ を繰り返しています。この繰り返しの周期は $4$ です。

## 求める末桁
$100$ は $4$ の倍数なので、$7^{100}$ の末桁は繰り返しの周期である $1$ です。

## 答え
よって、$7^{100}$ の末桁は $1$ です。

## 解の出力
求める $7^{100}$ の末桁は $1$ です。よって、答えは $\boxed{1}$ です。
Extracted answer: 1
最終解答: 1
メモリを解放しました


# アプリ

##環境構築

In [1]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [2]:
import streamlit as st
from pyngrok import ngrok

In [3]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [4]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

## 実行ファイル

### version1（単純な回答）

#### version1.0

In [9]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice

# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# StreamlitアプリのUI
st.title('Math Problem Solver')
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')

if st.button('Solve'):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    question_type_counts = {}
    starting_counts = (2, 3)
    i = 0
    temperature = 0.9
    top_p = 1.0

    text_answers, code_answers = question_type_counts.get(i, starting_counts)
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    print(prompt)

    model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    generation_output = model.generate(**model_inputs,
                                       max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                       do_sample=True,
                                       temperature=temperature,
                                       top_p=top_p,
                                       num_return_sequences=1)

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    st.write(generated_text)

Overwriting streamlit_app.py


#### version1.1(関数化)


In [5]:
%%writefile utils.py
import torch
import os
import plotly.graph_objects as go
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# メモリ使用状況を確認する関数
def check_memory():
    gpu_memory, gpu_reserved, gpu_allocated, gpu_free = 0, 0, 0, 0
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated

    total_memory = int(os.popen('free -m').readlines()[1].split()[1]) / 1024
    used_memory = int(os.popen('free -m').readlines()[1].split()[2]) / 1024
    free_memory = total_memory - used_memory

    return {
        "gpu_memory": gpu_memory,
        "gpu_reserved": gpu_reserved,
        "gpu_allocated": gpu_allocated,
        "gpu_free": gpu_free,
        "total_memory": total_memory,
        "used_memory": used_memory,
        "free_memory": free_memory,
    }

# メモリ使用量を円グラフで表示する関数
def display_memory_usage(memory_info, title):
    fig = go.Figure(data=[go.Pie(labels=['Used Memory', 'Free Memory'],
                                 values=[memory_info['used_memory'], memory_info['free_memory']],
                                 title=title, hole=.3)])
    fig.update_layout(
        annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)]
    )
    return fig

# メモリ使用量のグラフを表示および更新する関数
def update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart):
    fig_cpu.data[0].values = [memory_info['used_memory'], memory_info['free_memory']]
    fig_cpu.update_layout(annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    cpu_chart.plotly_chart(fig_cpu, use_container_width=True)

    fig_gpu.data[0].values = [memory_info['gpu_allocated'], memory_info['gpu_free']]
    fig_gpu.update_layout(annotations=[dict(text=f"Total: {memory_info['gpu_memory']:.2f} GB<br>Used: {memory_info['gpu_allocated']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    gpu_chart.plotly_chart(fig_gpu, use_container_width=True)

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        st.write("GPUが使用可能です。")
        st.write(f"使用中のGPU: {gpu_name}")
        return torch.device("cuda")
    else:
        st.write("GPUが使用できません。CPUを使用しますか？")
        if st.button("CPUを使用して続行"):
            return torch.device("cpu")
        else:
            st.write("処理を中止します。")
            return None

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path, device):
    st.write("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    with st.spinner('モデルをダウンロードしています...'):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype="auto",
            trust_remote_code=True,
            config=config
        )

    # モデルを適切なデバイス（GPUまたはCPU）に移動
    model = model.to(device)
    st.write("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

def generate_response(prompt, model, tokenizer, device):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    temperature = 0.9
    top_p = 1.0
    model.to(device)
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=1
    )

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

Writing utils.py


In [6]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
from numpy.random import choice
from utils import (
    check_memory,
    display_memory_usage,
    update_memory_charts,
    check_gpu,
    load_model_and_tokenizer,
    generate_response
)

# メモリの確認と可視化
memory_info = check_memory()

# CPUとGPUのメモリ使用状況を表示
col1, col2 = st.columns(2)
fig_cpu = display_memory_usage({
    "total_memory": memory_info['total_memory'],
    "used_memory": memory_info['used_memory'],
    "free_memory": memory_info['free_memory']
}, "CPU Memory Usage")
fig_gpu = display_memory_usage({
    "total_memory": memory_info['gpu_memory'],
    "used_memory": memory_info['gpu_allocated'],
    "free_memory": memory_info['gpu_free']
}, "GPU Memory Usage")

with col1:
    cpu_chart = st.plotly_chart(fig_cpu, use_container_width=True)
with col2:
    gpu_chart = st.plotly_chart(fig_gpu, use_container_width=True)

# GPUの確認
device = check_gpu()
if device is None:
    st.stop()

MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
model, tokenizer = load_model_and_tokenizer(MODEL_PATH, device)

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# StreamlitアプリのUI
st.title('Math Problem Solver')

# 問題の入力と解決
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
if st.button('Solve'):
    st.write("問題を読み込みました。")
    st.write(f"問題文はこれです: {problem}")

    text_answers = 2
    code_answers = 3
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    st.write("1回目の思考に入ります。プロンプト:")
    st.write(prompt)

    generated_text = generate_response(prompt, model, tokenizer, device)
    st.write("生成された回答:")
    st.write(generated_text)

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

Writing streamlit_app.py


In [ ]:
# 問題の入力と解決
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
if st.button('Solve'):
    st.write("問題を読み込みました。")
    st.write(f"問題文はこれです: {problem}")

    text_answers = 2
    code_answers = 3
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    st.write("1回目の思考に入ります。プロンプト:")
    st.write(prompt)

    generated_text = generate_response(prompt, model, tokenizer, device)
    st.write("生成された回答:")
    st.write(generated_text)

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

In [12]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice
import time
import gc
import os
import plotly.graph_objects as go

# メモリ使用状況を確認する関数
def check_memory():
    gpu_memory, gpu_reserved, gpu_allocated, gpu_free = 0, 0, 0, 0
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated

    total_memory = int(os.popen('free -m').readlines()[1].split()[1]) / 1024
    used_memory = int(os.popen('free -m').readlines()[1].split()[2]) / 1024
    free_memory = total_memory - used_memory

    return {
        "gpu_memory": gpu_memory,
        "gpu_reserved": gpu_reserved,
        "gpu_allocated": gpu_allocated,
        "gpu_free": gpu_free,
        "total_memory": total_memory,
        "used_memory": used_memory,
        "free_memory": free_memory,
    }

# メモリ使用量を円グラフで表示する関数
def display_memory_usage(memory_info):
    labels = ['Used Memory', 'Free Memory']
    values = [memory_info['used_memory'], memory_info['free_memory']]
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, title="CPU Memory Usage")])
    st.plotly_chart(fig)

    if torch.cuda.is_available():
        labels_gpu = ['Allocated Memory', 'Free Memory']
        values_gpu = [memory_info['gpu_allocated'], memory_info['gpu_free']]
        fig_gpu = go.Figure(data=[go.Pie(labels=labels_gpu, values=values_gpu, title="GPU Memory Usage")])
        st.plotly_chart(fig_gpu)

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        st.write("GPUが使用可能です。")
        st.write(f"使用中のGPU: {gpu_name}")
        return torch.device("cuda")
    else:
        st.write("GPUが使用できません。CPUを使用しますか？")
        if st.button("CPUを使用して続行"):
            return torch.device("cpu")
        else:
            st.write("処理を中止します。")
            return None

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path):
    st.write("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    with st.spinner('モデルをダウンロードしています...'):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype="auto",
            trust_remote_code=True,
            config=config
        )
    st.write("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

def generate_response(prompt, model, tokenizer, device):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    temperature = 0.9
    top_p = 1.0

    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    generation_output = model.generate(**model_inputs,
                                       max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                       do_sample=True,
                                       temperature=temperature,
                                       top_p=top_p,
                                       num_return_sequences=1)

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

# StreamlitアプリのUI
st.title('Math Problem Solver')

# メモリの確認と可視化
memory_info = check_memory()
display_memory_usage(memory_info)

# GPUの確認
device = check_gpu()
if device is None:
    st.stop()

# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
model, tokenizer = load_model_and_tokenizer(MODEL_PATH)
model = model.to(device)  # モデルを適切なデバイスに移動

# 問題の入力と解決
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
if st.button('Solve'):
    st.write("問題を読み込みました。")
    st.write(f"問題文はこれです: {problem}")

    text_answers = 2
    code_answers = 3
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    st.write("1回目の思考に入ります。プロンプト:")
    st.write(prompt)

    generated_text = generate_response(prompt, model, tokenizer, device)
    st.write("1回目の生成結果:")
    st.write(generated_text)

    # 生成結果を解析し、追加生成が必要か判断
    if "incomplete" in generated_text:
        st.write("追加生成が必要です。")
        more_text = generate_response(prompt + generated_text, model, tokenizer, device)
        st.write("追加生成結果:")
        st.write(more_text)
    else:
        st.write("最終生成結果:")
        st.write(generated_text)

Overwriting streamlit_app.py


In [5]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice
import time
import gc
import os
import plotly.graph_objects as go

# メモリ使用状況を確認する関数
def check_memory():
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated
        st.write(f"GPUメモリ: {gpu_memory:.2f} GB")
        st.write(f"使用中のメモリ: {gpu_allocated:.2f} GB")
        st.write(f"予約済みのメモリ: {gpu_reserved:.2f} GB")
        st.write(f"利用可能なメモリ: {gpu_free:.2f} GB")
    else:
        st.write("GPUは利用できません。")

    total_memory = os.popen('free -m').readlines()[1].split()[1]
    used_memory = os.popen('free -m').readlines()[1].split()[2]
    st.write(f"CPUメモリ: {total_memory} MB")
    st.write(f"使用中のメモリ: {used_memory} MB")

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        st.write("GPUが使用可能です。")
        st.write(f"使用中のGPU: {gpu_name}")
        return torch.device("cuda")
    else:
        st.write("GPUが使用できません。CPUを使用しますか？")
        if st.button("CPUを使用して続行"):
            return torch.device("cpu")
        else:
            st.write("処理を中止します。")
            return None

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path):
    st.write("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    with st.spinner('モデルをダウンロードしています...'):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype="auto",
            trust_remote_code=True,
            config=config
        )
    st.write("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

def generate_response(prompt, model, tokenizer, device):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    temperature = 0.9
    top_p = 1.0

    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    generation_output = model.generate(**model_inputs,
                                       max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                       do_sample=True,
                                       temperature=temperature,
                                       top_p=top_p,
                                       num_return_sequences=1)

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

# StreamlitアプリのUI
st.title('Math Problem Solver')

# メモリの確認
check_memory()

# GPUの確認
device = check_gpu()
if device is None:
    st.stop()

# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
model, tokenizer = load_model_and_tokenizer(MODEL_PATH)
model = model.to(device)  # モデルを適切なデバイスに移動

# 問題の入力と解決
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
if st.button('Solve'):
    st.write("問題を読み込みました。")
    st.write(f"問題文はこれです: {problem}")

    text_answers = 2
    code_answers = 3
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    st.write("1回目の思考に入ります。プロンプト:")
    st.write(prompt)

    generated_text = generate_response(prompt, model, tokenizer, device)
    st.write("1回目の生成結果:")
    st.write(generated_text)

    # 生成結果を解析し、追加生成が必要か判断
    if "incomplete" in generated_text:
        st.write("追加生成が必要です。")
        more_text = generate_response(prompt + generated_text, model, tokenizer, device)
        st.write("追加生成結果:")
        st.write(more_text)
    else:
        st.write("最終生成結果:")
        st.write(generated_text)


Writing streamlit_app.py


#### version1.2（モジュール化）

In [ ]:
%%writefile utils.py
import torch
import os
import plotly.graph_objects as go
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# メモリ使用状況を確認する関数
def check_memory():
    gpu_memory, gpu_reserved, gpu_allocated, gpu_free = 0, 0, 0, 0
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated

    total_memory = int(os.popen('free -m').readlines()[1].split()[1]) / 1024
    used_memory = int(os.popen('free -m').readlines()[1].split()[2]) / 1024
    free_memory = total_memory - used_memory

    return {
        "gpu_memory": gpu_memory,
        "gpu_reserved": gpu_reserved,
        "gpu_allocated": gpu_allocated,
        "gpu_free": gpu_free,
        "total_memory": total_memory,
        "used_memory": used_memory,
        "free_memory": free_memory,
    }

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# メモリ使用量を円グラフで表示する関数
def display_memory_usage(memory_info, title):
    fig = go.Figure(data=[go.Pie(labels=['Used Memory', 'Free Memory'],
                                 values=[memory_info['used_memory'], memory_info['free_memory']],
                                 title=title, hole=.3)])
    fig.update_layout(
        annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)]
    )
    return fig

# メモリ使用量のグラフを表示および更新する関数
def update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart):
    fig_cpu.data[0].values = [memory_info['used_memory'], memory_info['free_memory']]
    fig_cpu.update_layout(annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    cpu_chart.plotly_chart(fig_cpu, use_container_width=True)

    fig_gpu.data[0].values = [memory_info['gpu_allocated'], memory_info['gpu_free']]
    fig_gpu.update_layout(annotations=[dict(text=f"Total: {memory_info['gpu_memory']:.2f} GB<br>Used: {memory_info['gpu_allocated']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    gpu_chart.plotly_chart(fig_gpu, use_container_width=True)

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        st.write("GPUが使用可能です。")
        st.write(f"使用中のGPU: {gpu_name}")
        return torch.device("cuda")
    else:
        st.write("GPUが使用できません。CPUを使用しますか？")
        if st.button("CPUを使用して続行"):
            return torch.device("cpu")
        else:
            st.write("処理を中止します。")
            return None

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path, device):
    st.write("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    with st.spinner('モデルをダウンロードしています...'):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype="auto",
            trust_remote_code=True,
            config=config
        )

    # モデルを適切なデバイス（GPUまたはCPU）に移動
    model = model.to(device)
    st.write("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

def generate_response(prompt, model, tokenizer, device):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    temperature = 0.9
    top_p = 1.0
    model.to(device)
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    # メモリの解放
    free_memory()
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=1
    )

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

Writing utils.py


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
from numpy.random import choice
from utils import (
    check_memory,
    display_memory_usage,
    update_memory_charts,
    check_gpu,
    load_model_and_tokenizer,
    generate_response
)

# メモリの確認と可視化
memory_info = check_memory()

# CPUとGPUのメモリ使用状況を表示
col1, col2 = st.columns(2)
fig_cpu = display_memory_usage({
    "total_memory": memory_info['total_memory'],
    "used_memory": memory_info['used_memory'],
    "free_memory": memory_info['free_memory']
}, "CPU Memory Usage")
fig_gpu = display_memory_usage({
    "total_memory": memory_info['gpu_memory'],
    "used_memory": memory_info['gpu_allocated'],
    "free_memory": memory_info['gpu_free']
}, "GPU Memory Usage")

with col1:
    cpu_chart = st.plotly_chart(fig_cpu, use_container_width=True)
with col2:
    gpu_chart = st.plotly_chart(fig_gpu, use_container_width=True)

# GPUの確認
device = check_gpu()
if device is None:
    st.stop()

MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
model, tokenizer = load_model_and_tokenizer(MODEL_PATH, device)

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# StreamlitアプリのUI
st.title('Math Problem Solver')

# 問題の入力と解決
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
if st.button('Solve'):
    st.write("問題を読み込みました。")
    st.write(f"問題文はこれです: {problem}")

    text_answers = 2
    code_answers = 3
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    st.write("1回目の思考に入ります。プロンプト:")
    st.write(prompt)

    generated_text = generate_response(prompt, model, tokenizer, device)
    st.write("生成された回答:")
    st.write(generated_text)

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

Writing streamlit_app.py


### version2

#### version2.1

In [8]:
%%writefile predict.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice
import time
import gc
from collections import Counter
from tqdm import tqdm

extra_time = 0
allowed = 0
TOTAL_TOKENS = 2048
USE_PAST_KEY = False
NOTEBOOK_START_TIME = time.time()
# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

def process_code(code, return_shell_output=False):
    # Dummy implementation, replace with actual code processing logic
    return -1, False

def process_text_output(raw_output):
    # Dummy implementation, replace with actual text processing logic
    return -1

def predict(problem, model, tokenizer):
    seed_everything(69)

    temperature = 0.9
    top_p = 1.0

    temperature_coding = 0.9
    top_p_coding = 1.0

    total_results = {}
    total_answers = {}
    best_stats = {}
    total_outputs = {}
    question_type_counts = {}
    starting_counts = (2, 3)
    i = 0

    global extra_time, allowed, NOTEBOOK_START_TIME, promplt_options, TOTAL_TOKENS

    if time.time() - NOTEBOOK_START_TIME >= 32200:
        return 0

    PROBLEM_START_TIME = time.time()
    allowed = min(extra_time, 600)

    for jj in tqdm(range(5)):  # n_repetitionsを5に設定
        best, best_count = best_stats.get(i, (-1, -1))
        if best_count > np.sqrt(jj):
            print("SKIPPING CAUSE ALREADY FOUND BEST")
            continue

        if time.time() - PROBLEM_START_TIME >= 720 + allowed:
            extra = (time.time() - PROBLEM_START_TIME) - 720
            extra_time -= max(0, extra)
            return best_stats[0][0]

        if time.time() - NOTEBOOK_START_TIME >= 32200:
            return best_stats[0][0]

        outputs = total_outputs.get(i, [])
        text_answers, code_answers = question_type_counts.get(i, starting_counts)
        results = total_results.get(i, [])
        answers = total_answers.get(i, [])

        for _ in range(5):
            torch.cuda.empty_cache()
            gc.collect()
            time.sleep(0.2)

        try:
            ALREADY_GEN = 0
            code_error = None
            code_error_count = 0
            code_output = -1
            counts = np.array([text_answers, code_answers])

            draw = choice(promplt_options, 1, p=counts / counts.sum())
            initial_message = draw[0].format(problem, "{}")
            prompt = f"User: {initial_message}"

            current_printed = len(prompt)
            print(f"{jj}_{prompt}\n")

            model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
            input_len = len(model_inputs['input_ids'][0])

            generation_output = model.generate(
                **model_inputs,
                max_new_tokens=TOTAL_TOKENS - ALREADY_GEN,
                return_dict_in_generate=USE_PAST_KEY,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                num_return_sequences=1,
            )

            if USE_PAST_KEY:
                output_ids = generation_output.sequences[0]
            else:
                output_ids = generation_output[0]
            decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
            print(f"{decoded_output[current_printed:]}\n")
            current_printed += len(decoded_output[current_printed:])
            cummulative_code = ""

            stop_word_cond = False
            for stop_word in ["your", "stop", "words"]:
                stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):] == stop_word)

            while stop_word_cond and ALREADY_GEN < TOTAL_TOKENS:
                if decoded_output[-len("```python"):] == "```python":
                    temperature_inner = temperature_coding
                    top_p_inner = top_p_coding
                    prompt = decoded_output
                else:
                    temperature_inner = temperature
                    top_p_inner = top_p
                    try:
                        if decoded_output[-len("``````output"):] == "``````output":
                            code_text = decoded_output.split('```python')[-1].split("``````")[0]
                        else:
                            code_text = decoded_output.split('```python')[-1].split("```")[0]

                        cummulative_code += code_text
                        code_output, CODE_STATUS = process_code(cummulative_code, return_shell_output=True)
                        print('CODE RESULTS', code_output)

                        if code_error == code_output:
                            code_error_count += 1
                        else:
                            code_error = code_output
                            code_error_count = 0

                        if not CODE_STATUS:
                            cummulative_code = cummulative_code[:-len(code_text)]

                            if code_error_count >= 1:
                                print("REPEATED ERRORS")
                                break

                    except Exception as e:
                        print(e)
                        print('ERROR PARSING CODE')
                        code_output = -1

                    if code_output != -1:
                        if decoded_output[-len(")\n```"):] == ")\n```":
                            prompt = decoded_output + '```output\n' + str(code_output) + '\n```\n'
                        else:
                            prompt = decoded_output + '\n' + str(code_output) + '\n```\n'
                    else:
                        prompt = decoded_output
                        cummulative_code = ""
                model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
                ALREADY_GEN = len(model_inputs['input_ids'][0]) - input_len

                if USE_PAST_KEY:
                    old_values = generation_output.past_key_values
                else:
                    old_values = None

                generation_output = model.generate(
                    **model_inputs,
                    max_new_tokens=TOTAL_TOKENS - ALREADY_GEN,
                    return_dict_in_generate=USE_PAST_KEY,
                    past_key_values=old_values,
                    do_sample=True,
                    temperature=temperature_inner,
                    top_p=top_p_inner,
                    num_return_sequences=1
                )
                if USE_PAST_KEY:
                    output_ids = generation_output.sequences[0]
                else:
                    output_ids = generation_output[0]
                decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
                print(f"\nINTERMEDIATE OUT :\n{decoded_output[current_printed:]}\n")
                current_printed += len(decoded_output[current_printed:])

                stop_word_cond = False
                for stop_word in ["your", "stop", "words"]:
                    stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):] == stop_word)
            if USE_PAST_KEY:
                output_ids = generation_output.sequences[0]
            else:
                output_ids = generation_output[0]

            raw_output = tokenizer.decode(output_ids[input_len:], skip_special_tokens=True)
            result_output = process_text_output(raw_output)

            try:
                code_output = round(float(eval(code_output))) % 1000
            except Exception as e:
                print(e, 'final_eval')
                code_output = -1
        except Exception as e:
            print(e, "5")
            result_output, code_output = -1, -1

        if code_output != -1:
            outputs.append(code_output)
            code_answers += 1

        if result_output != -1:
            outputs.append(result_output)
            text_answers += 1

        if len(outputs) > 0:
            occurances = Counter(outputs).most_common()
            print(occurances)
            if occurances[0][1] > best_count:
                print("GOOD ANSWER UPDATED!")
                best = occurances[0][0]
                best_count = occurances[0][1]
            if occurances[0][1] > 5:
                print("ANSWER FOUND!")
                break

        results.append(result_output)
        answers.append(code_output)

        best_stats[i] = (best, best_count)
        question_type_counts[i] = (text_answers, code_answers)
        total_outputs[i] = outputs

        total_results[i] = results
        total_answers[i] = answers

        print("code_answers", code_answers - starting_counts[1], "text_answers", text_answers - starting_counts[0])
    remaining = 720 - (time.time() - PROBLEM_START_TIME)
    remaining = max(0, remaining)
    extra_time += remaining
    return best_stats[0][0]

def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

Writing predict.py


In [9]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice
from predict import predict

# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"  # より軽量なモデルに変更
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, config=config)

# StreamlitアプリのUI
st.title('Pathway Solver AI')
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')

if st.button('Solve'):
    result = predict(problem, model, tokenizer)
    st.write(result)

Overwriting streamlit_app.py


#### version2.2

In [ ]:
%%writefile utils.py
import torch
import os
import gc
import plotly.graph_objects as go
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# メモリ使用状況を確認する関数
def check_memory():
    gpu_memory, gpu_reserved, gpu_allocated, gpu_free = 0, 0, 0, 0
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated

    total_memory = int(os.popen('free -m').readlines()[1].split()[1]) / 1024
    used_memory = int(os.popen('free -m').readlines()[1].split()[2]) / 1024
    free_memory = total_memory - used_memory

    return {
        "gpu_memory": gpu_memory,
        "gpu_reserved": gpu_reserved,
        "gpu_allocated": gpu_allocated,
        "gpu_free": gpu_free,
        "total_memory": total_memory,
        "used_memory": used_memory,
        "free_memory": free_memory,
    }

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# メモリ使用量を円グラフで表示する関数
def display_memory_usage(memory_info, title):
    fig = go.Figure(data=[go.Pie(labels=['Used Memory', 'Free Memory'],
                                 values=[memory_info['used_memory'], memory_info['free_memory']],
                                 title=title, hole=.3)])
    fig.update_layout(
        annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)]
    )
    return fig

# メモリ使用量のグラフを表示および更新する関数
def update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart):
    fig_cpu.data[0].values = [memory_info['used_memory'], memory_info['free_memory']]
    fig_cpu.update_layout(annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    cpu_chart.plotly_chart(fig_cpu, use_container_width=True)

    fig_gpu.data[0].values = [memory_info['gpu_allocated'], memory_info['gpu_free']]
    fig_gpu.update_layout(annotations=[dict(text=f"Total: {memory_info['gpu_memory']:.2f} GB<br>Used: {memory_info['gpu_allocated']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    gpu_chart.plotly_chart(fig_gpu, use_container_width=True)

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        return torch.device("cuda"), gpu_name
    else:
        return torch.device("cpu"), "CPU"

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path, device, st):
    st.write("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    with st.spinner('モデルをダウンロードしています...'):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype="auto",
            trust_remote_code=True,
            config=config
        )

    # モデルを適切なデバイス（GPUまたはCPU）に移動
    model = model.to(device)
    st.write("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

def generate_response(prompt, model, tokenizer, device):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    temperature = 0.9
    top_p = 1.0
    model.to(device)
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    # メモリの解放
    free_memory()
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=1
    )

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

Overwriting utils.py


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
from numpy.random import choice
from utils import (
    check_memory,
    display_memory_usage,
    update_memory_charts,
    check_gpu,
    load_model_and_tokenizer,
    generate_response,
    free_memory
)

# StreamlitアプリのUI
st.title('Math Problem Solver')

# メモリの確認と可視化
memory_info = check_memory()

# CPUとGPUのメモリ使用状況を表示
col1, col2 = st.columns(2)
fig_cpu = display_memory_usage({
    "total_memory": memory_info['total_memory'],
    "used_memory": memory_info['used_memory'],
    "free_memory": memory_info['free_memory']
}, "CPU Memory Usage")
fig_gpu = display_memory_usage({
    "total_memory": memory_info['gpu_memory'],
    "used_memory": memory_info['gpu_allocated'],
    "free_memory": memory_info['gpu_free']
}, "GPU Memory Usage")

with col1:
    cpu_chart = st.plotly_chart(fig_cpu, use_container_width=True)
with col2:
    gpu_chart = st.plotly_chart(fig_gpu, use_container_width=True)

# GPUの確認
device, gpu_name = check_gpu()
if device is None:
    st.stop()

st.write(f"使用中のデバイス: {gpu_name}")

MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
model, tokenizer = load_model_and_tokenizer(MODEL_PATH, device, st)

# プロンプトテンプレート
code_template = """以下の数学の問題を解いてください（正の数値で回答）:
"{}"
まず、問題を解決するための手順を列挙し、それぞれの手順で呼び出す関数を示してください。全てのステップをコメントで丁寧に記述し、最終的な答えを \\boxed{} 内に出力してください。

アプローチ:"""

cot_template = """以下の数学の問題を解いてください（正の数値で回答）:
"{}"
問題を分析し、ステップごとに解決策を考えてください。解決後、最終的な答えを \\boxed{} 内に出力してください。

"""

prompt_options = [code_template, cot_template]

# 問題の入力と解決
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
if st.button('Solve'):
    st.write("問題を読み込みました。")
    st.write(f"問題文はこれです: {problem}")

    text_answers = 2
    code_answers = 3
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    st.write("1回目の思考に入ります。プロンプト:")
    st.write(prompt)

    generated_text = generate_response(prompt, model, tokenizer, device)
    st.write("生成された回答:")
    st.write(generated_text)

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

    # モデルのメモリ解放
    free_memory()
    del model

Overwriting streamlit_app.py


In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

#### version2.3(メモリを解放するだけ)

In [12]:
%%writefile utils.py

import torch
import os
import gc
import plotly.graph_objects as go
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# メモリ使用状況を確認する関数
def check_memory():
    gpu_memory, gpu_reserved, gpu_allocated, gpu_free = 0, 0, 0, 0
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated

    total_memory = int(os.popen('free -m').readlines()[1].split()[1]) / 1024
    used_memory = int(os.popen('free -m').readlines()[1].split()[2]) / 1024
    free_memory = total_memory - used_memory

    return {
        "gpu_memory": gpu_memory,
        "gpu_reserved": gpu_reserved,
        "gpu_allocated": gpu_allocated,
        "gpu_free": gpu_free,
        "total_memory": total_memory,
        "used_memory": used_memory,
        "free_memory": free_memory,
    }

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# メモリ使用量を円グラフで表示する関数
def display_memory_usage(memory_info, title):
    fig = go.Figure(data=[go.Pie(labels=['Used Memory', 'Free Memory'],
                                 values=[memory_info['used_memory'], memory_info['free_memory']],
                                 title=title, hole=.3)])
    fig.update_layout(
        annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)]
    )
    return fig

# メモリ使用量のグラフを表示および更新する関数
def update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart):
    fig_cpu.data[0].values = [memory_info['used_memory'], memory_info['free_memory']]
    fig_cpu.update_layout(annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    cpu_chart.plotly_chart(fig_cpu, use_container_width=True)

    fig_gpu.data[0].values = [memory_info['gpu_allocated'], memory_info['gpu_free']]
    fig_gpu.update_layout(annotations=[dict(text=f"Total: {memory_info['gpu_memory']:.2f} GB<br>Used: {memory_info['gpu_allocated']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    gpu_chart.plotly_chart(fig_gpu, use_container_width=True)

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        return torch.device("cuda"), gpu_name
    else:
        return torch.device("cpu"), "CPU"

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path, device, st):
    st.write("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    with st.spinner('モデルをダウンロードしています...'):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype="auto",
            trust_remote_code=True,
            config=config
        )

    # モデルを適切なデバイス（GPUまたはCPU）に移動
    model = model.to(device)
    st.write("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

def generate_response(prompt, model, tokenizer, device):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    temperature = 0.9
    top_p = 1.0
    model.to(device)
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    # メモリの解放
    free_memory()
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=1
    )

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

Writing utils.py


In [13]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
from numpy.random import choice
from utils import (
    check_memory,
    display_memory_usage,
    update_memory_charts,
    check_gpu,
    load_model_and_tokenizer,
    free_memory
)

# メモリの確認と可視化
memory_info = check_memory()

# CPUとGPUのメモリ使用状況を表示
col1, col2 = st.columns(2)
fig_cpu = display_memory_usage({
    "total_memory": memory_info['total_memory'],
    "used_memory": memory_info['used_memory'],
    "free_memory": memory_info['free_memory']
}, "CPU Memory Usage")
fig_gpu = display_memory_usage({
    "total_memory": memory_info['gpu_memory'],
    "used_memory": memory_info['gpu_allocated'],
    "free_memory": memory_info['gpu_free']
}, "GPU Memory Usage")

with col1:
    cpu_chart = st.plotly_chart(fig_cpu, use_container_width=True)
with col2:
    gpu_chart = st.plotly_chart(fig_gpu, use_container_width=True)

# GPUの確認
device, gpu_name = check_gpu()
if device is None:
    st.stop()

st.write(f"使用中のデバイス: {gpu_name}")

MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"

# モデルの読み込みボタン
if st.button('モデルを読み込む'):
    model, tokenizer = load_model_and_tokenizer(MODEL_PATH, device, st)
    st.session_state['model'] = model
    st.session_state['tokenizer'] = tokenizer

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

# メモリ解放ボタン
if st.button('メモリを解放'):
    if 'model' in st.session_state:
        del st.session_state['model']
    if 'tokenizer' in st.session_state:
        del st.session_state['tokenizer']
    free_memory()

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

Overwriting streamlit_app.py


### version3(2回確認)

#### version3.1(ノーマル)

In [5]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice
import re

# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# StreamlitアプリのUI
st.title('Math Problem Solver')
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')

def evaluate_answer(answer):
    # 答えを評価するロジック（ここでは簡単に出力をパースして数値を探す）
    match = re.search(r'\\boxed{(\d+)}', answer)
    if match:
        return int(match.group(1))
    else:
        return None

def generate_prompt(problem, past_output=None):
    # 過去の出力を考慮してプロンプトを生成する
    if past_output:
        new_prompt = f"Previous attempt:\n{past_output}\n\n"
        new_prompt += "Analyze the previous attempt and correct any mistakes to solve the problem: \n"
    else:
        text_answers, code_answers = question_type_counts.get(i, starting_counts)
        counts = np.array([text_answers, code_answers])
        draw = choice(prompt_options, 1, p=counts/counts.sum())
        new_prompt = draw[0].format(problem, "{}")
    return f"User: {new_prompt}"

if st.button('Solve'):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    question_type_counts = {}
    starting_counts = (2, 3)
    i = 0
    temperature = 0.9
    top_p = 1.0

    initial_prompt = generate_prompt(problem)
    prompt = initial_prompt
    print(prompt)

    model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    generation_output = model.generate(**model_inputs,
                                       max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                       do_sample=True,
                                       temperature=temperature,
                                       top_p=top_p,
                                       num_return_sequences=1)

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    st.write(generated_text)

    answer = evaluate_answer(generated_text)
    if answer is None:
        # 初回の回答が望ましい結果でない場合、次のプロンプトを生成
        st.write("Re-evaluating the response...")
        additional_prompt = generate_prompt(problem, generated_text)
        model_inputs = tokenizer(additional_prompt, return_tensors='pt').to(model.device)
        generation_output = model.generate(**model_inputs,
                                           max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                           do_sample=True,
                                           temperature=temperature,
                                           top_p=top_p,
                                           num_return_sequences=1)
        generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
        st.write(generated_text)
    else:
        st.write(f"Final Answer: {answer}")

Writing streamlit_app.py


#### version3.2(メモリ解法)

In [16]:
%%writefile utils.py
import torch
import os
import gc
import plotly.graph_objects as go
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import re
import numpy as np
from numpy.random import choice

# メモリ使用状況を確認する関数
def check_memory():
    gpu_memory, gpu_reserved, gpu_allocated, gpu_free = 0, 0, 0, 0
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        gpu_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        gpu_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        gpu_free = gpu_memory - gpu_allocated

    total_memory = int(os.popen('free -m').readlines()[1].split()[1]) / 1024
    used_memory = int(os.popen('free -m').readlines()[1].split()[2]) / 1024
    free_memory = total_memory - used_memory

    return {
        "gpu_memory": gpu_memory,
        "gpu_reserved": gpu_reserved,
        "gpu_allocated": gpu_allocated,
        "gpu_free": gpu_free,
        "total_memory": total_memory,
        "used_memory": used_memory,
        "free_memory": free_memory,
    }

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# メモリ使用量を円グラフで表示する関数
def display_memory_usage(memory_info, title):
    fig = go.Figure(data=[go.Pie(labels=['Used Memory', 'Free Memory'],
                                 values=[memory_info['used_memory'], memory_info['free_memory']],
                                 title=title, hole=.3)])
    fig.update_layout(
        annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)]
    )
    return fig

# メモリ使用量のグラフを表示および更新する関数
def update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart):
    fig_cpu.data[0].values = [memory_info['used_memory'], memory_info['free_memory']]
    fig_cpu.update_layout(annotations=[dict(text=f"Total: {memory_info['total_memory']:.2f} GB<br>Used: {memory_info['used_memory']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    cpu_chart.plotly_chart(fig_cpu, use_container_width=True)

    fig_gpu.data[0].values = [memory_info['gpu_allocated'], memory_info['gpu_free']]
    fig_gpu.update_layout(annotations=[dict(text=f"Total: {memory_info['gpu_memory']:.2f} GB<br>Used: {memory_info['gpu_allocated']:.2f} GB",
                          x=0.5, y=0.5, font_size=12, showarrow=False)])
    gpu_chart.plotly_chart(fig_gpu, use_container_width=True)

# GPUの使用可否と種類を確認
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        return torch.device("cuda"), gpu_name
    else:
        return torch.device("cpu"), "CPU"

# モデルとトークナイザーの読み込み
def load_model_and_tokenizer(model_path, device, st):
    st.write("モデルとトークナイザーを読み込んでいます...")
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # モデルを読み込む際に進捗を表示
    with st.spinner('モデルをダウンロードしています...'):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype="auto",
            trust_remote_code=True,
            config=config
        )

    # モデルを適切なデバイス（GPUまたはCPU）に移動
    model = model.to(device)
    st.write("モデルとトークナイザーの読み込みが完了しました。")
    return model, tokenizer

def generate_response(prompt, model, tokenizer, device):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    temperature = 0.9
    top_p = 1.0
    model.to(device)
    model_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    # メモリの解放
    free_memory()
    generation_output = model.generate(
        **model_inputs,
        max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=1
    )

    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    return generated_text

def evaluate_answer(answer):
    # 答えを評価するロジック（ここでは簡単に出力をパースして数値を探す）
    match = re.search(r'\\boxed{(\d+)}', answer)
    if match:
        return int(match.group(1))
    else:
        return None

def generate_prompt(problem, past_output=None):
    # プロンプトテンプレート
    code_template = """以下は解くべき数学の問題です（正の数値解を求めてください）:
「{}」
これを解決するために、まずSymPyを使った解法を考え、それぞれのステップで呼び出す必要のある関数をリストアップしてください。誰でも理解できるように明確にし、最終的な答えは代数式ではなく正の整数にしてください！
すべてのステップをカバーするスクリプトを書いてください（コメントとドキュメントを含む）結果を表示してください。問題を解いた後、最終的な数値の答えを\\boxed{}で囲んで出力してください。

アプローチ:"""

    cot_template = """以下は解くべき数学の問題です（正の数値解を求めてください！）:
「{}」
この問題を分析し、プログラムを使って段階的に解決策を考えてください。問題を解いた後、最終的な数値の答えを\\boxed{}で囲んで出力してください。\n\n"""

    prompt_options = [code_template, cot_template]

    # 過去の出力を考慮してプロンプトを生成する
    if past_output:
        new_prompt = f"Previous attempt:\n{past_output}\n\n"
        new_prompt += "Analyze the previous attempt and correct any mistakes to solve the problem: \n"
    else:
        new_prompt = choice(prompt_options).format(problem, "{}")
    return f"User: {new_prompt}"


Overwriting utils.py


In [33]:
%%writefile streamlit_app.py
import streamlit as st
import time
from collections import Counter
from utils import (
    check_memory,
    display_memory_usage,
    update_memory_charts,
    check_gpu,
    load_model_and_tokenizer,
    free_memory,
    generate_response,
    evaluate_answer,
    generate_prompt
)

# メモリの確認と可視化
memory_info = check_memory()

# CPUとGPUのメモリ使用状況を表示
col1, col2 = st.columns(2)
fig_cpu = display_memory_usage({
    "total_memory": memory_info['total_memory'],
    "used_memory": memory_info['used_memory'],
    "free_memory": memory_info['free_memory']
}, "CPU Memory Usage")
fig_gpu = display_memory_usage({
    "total_memory": memory_info['gpu_memory'],
    "used_memory": memory_info['gpu_allocated'],
    "free_memory": memory_info['gpu_free']
}, "GPU Memory Usage")

with col1:
    cpu_chart = st.plotly_chart(fig_cpu, use_container_width=True)
with col2:
    gpu_chart = st.plotly_chart(fig_gpu, use_container_width=True)

# GPUの確認
device, gpu_name = check_gpu()
if device is None:
    st.stop()

st.write(f"使用中のデバイス: {gpu_name}")

MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"

# モデルの読み込みボタン
if st.button('モデルを読み込む'):
    model, tokenizer = load_model_and_tokenizer(MODEL_PATH, device, st)
    st.session_state['model'] = model
    st.session_state['tokenizer'] = tokenizer

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

# StreamlitアプリのUI
st.title('Math Problem Solver')
problem = st.text_input('解くべき数学の問題を入力してください:', 'Find the units digit of \( 7^{100} \).')
iterations = st.number_input('試行回数（1-100）:', min_value=1, max_value=100, value=3)
total_time_limit = st.number_input('合計時間制限（秒、10-3600）:', min_value=10, max_value=3600, value=300)
single_attempt_time_limit = st.number_input('1回の試行時間制限（秒、1-300）:', min_value=1, max_value=300, value=30)

if st.button('Solve'):
    if 'model' in st.session_state and 'tokenizer' in st.session_state:
        model = st.session_state['model']
        tokenizer = st.session_state['tokenizer']

        start_time = time.time()
        final_answer = None
        past_outputs = []
        answers = []

        initial_prompt = generate_prompt(problem)
        prompt = initial_prompt
        print(prompt)

        for attempt in range(iterations):
            if time.time() - start_time > total_time_limit:
                st.write("合計時間制限を超えました。")
                break

            attempt_start_time = time.time()
            while time.time() - attempt_start_time < single_attempt_time_limit:
                generated_text = generate_response(prompt, model, tokenizer, device)
                st.write(f"試行 {attempt + 1} の出力:")
                st.write(generated_text)

                past_outputs.append(generated_text)
                answer = evaluate_answer(generated_text)
                if answer is not None:
                    answers.append(answer)
                    break  # 有効な解答が見つかった場合、内側のループを抜ける

            if answers:
                final_answer = Counter(answers).most_common(1)[0][0]
                st.write(f"最終回答: {final_answer}")
                break
        else:
            if answers:
                final_answer = Counter(answers).most_common(1)[0][0]
                st.write(f"最終回答: {final_answer}")
            else:
                st.write("指定された試行回数内に有効な回答が見つかりませんでした。")
                if past_outputs:
                    valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
                    if valid_answers:
                        final_answer = choice(valid_answers, 1)[0]
                        st.write(f"過去の試行からランダムに選ばれた回答: {final_answer}")

# メモリ解放ボタン
if st.button('メモリを解放'):
    if 'model' in st.session_state:
        del st.session_state['model']
    if 'tokenizer' in st.session_state:
        del st.session_state['tokenizer']
    free_memory()

    # メモリ使用量の更新
    memory_info = check_memory()
    update_memory_charts(memory_info, fig_cpu, fig_gpu, cpu_chart, gpu_chart)

Overwriting streamlit_app.py


### vesion4(複数回)

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice
import re
import time
from collections import Counter

# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# StreamlitアプリのUI
st.title('Math Problem Solver')
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
iterations = st.number_input('Number of attempts:', min_value=1, max_value=10, value=1)

def evaluate_answer(answer):
    # 答えを評価するロジック（ここでは簡単に出力をパースして数値を探す）
    match = re.search(r'\\boxed{(\d+)}', answer)
    if match:
        return int(match.group(1))
    else:
        return None

def generate_prompt(problem, past_outputs=None):
    # 過去の出力を考慮してプロンプトを生成する
    if past_outputs:
        new_prompt = "Previous attempts:\n"
        for output in past_outputs:
            new_prompt += f"{output}\n\n"
        new_prompt += "Analyze the previous attempts and correct any mistakes to solve the problem: \n"
    else:
        text_answers, code_answers = question_type_counts.get(i, starting_counts)
        counts = np.array([text_answers, code_answers])
        draw = choice(prompt_options, 1, p=counts/counts.sum())
        new_prompt = draw[0].format(problem, "{}")
    return f"User: {new_prompt}"

if st.button('Solve'):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    question_type_counts = {}
    starting_counts = (2, 3)
    i = 0
    temperature = 0.9
    top_p = 1.0

    final_answer = None
    past_outputs = []
    answers = []
    initial_prompt = generate_prompt(problem)
    prompt = initial_prompt

    for attempt in range(iterations):
        print(f"Attempt {attempt + 1}")

        model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
        generation_output = model.generate(**model_inputs,
                                           max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                           do_sample=True,
                                           temperature=temperature,
                                           top_p=top_p,
                                           num_return_sequences=1)

        generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
        st.write(f"Attempt {attempt + 1} Output:")
        st.write(generated_text)

        past_outputs.append(generated_text)

        answer = evaluate_answer(generated_text)
        if answer is not None:
            answers.append(answer)
        else:
            st.write("Re-evaluating the response...")
            prompt = generate_prompt(problem, past_outputs)
            time.sleep(2)  # 少しの待機時間を追加して、次の試行に移る前に処理を安定させる

    if answers:
        final_answer = Counter(answers).most_common(1)[0][0]
        st.write(f"Final Answer: {final_answer}")
    else:
        st.write("Could not find a valid answer within the specified attempts.")
        if past_outputs:
            final_answer = choice([evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None], 1)[0]
            st.write(f"Randomly selected answer from past attempts: {final_answer}")

Overwriting streamlit_app.py


### version5(複数回、時間制限)

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice
import time
from collections import Counter

# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# 解答を評価する関数
def evaluate_answer(answer):
    try:
        return int(answer.strip())  # ダミーの評価: 数値に変換できれば有効とする
    except:
        return None

# プロンプトを生成する関数
def generate_prompt(problem, past_outputs=None):
    counts = np.array([2, 3])
    draw = choice(prompt_options, 1, p=counts/counts.sum())
    if past_outputs:
        previous_output = "\n".join(past_outputs)
        prompt = f"User: {draw[0].format(problem)}\n{previous_output}"
    else:
        prompt = f"User: {draw[0].format(problem)}"
    return prompt

# StreamlitアプリのUI設定
st.title('Math Problem Solver')
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')
iterations = st.number_input('Number of iterations:', min_value=1, max_value=100, value=10)
total_time_limit = st.number_input('Total time limit (seconds):', min_value=10, max_value=3600, value=300)
single_attempt_time_limit = st.number_input('Single attempt time limit (seconds):', min_value=1, max_value=300, value=30)

if st.button('Solve'):
    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []
    initial_prompt = generate_prompt(problem)
    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            st.write("Total time limit exceeded.")
            break

        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
            generation_output = model.generate(**model_inputs,
                                               max_new_tokens=2048,
                                               do_sample=True,
                                               temperature=0.9,
                                               top_p=1.0,
                                               num_return_sequences=1)

            generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
            st.write(f"Attempt {attempt + 1} Output:")
            st.write(generated_text)

            past_outputs.append(generated_text)

            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)
                break  # 有効な解答が見つかった場合、内側のループを抜ける

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延

        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            st.write(f"Final Answer: {final_answer}")
            break
    else:
        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            st.write(f"Final Answer: {final_answer}")
        else:
            st.write("Could not find a valid answer within the specified attempts.")
            if past_outputs:
                valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
                if valid_answers:
                    final_answer = choice(valid_answers, 1)[0]
                    st.write(f"Randomly selected answer from past attempts: {final_answer}")

Writing streamlit_app.py


'Find the units digit of \( 7^{100} \).'

### version6

In [ ]:
import time
import numpy as np
from numpy.random import choice
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
from collections import Counter

In [ ]:
# モデルとトークナイザーの読み込み
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

In [ ]:
a="あああ"
prompt_options[0].format(a)

IndexError: Replacement index 1 out of range for positional args tuple

In [ ]:
prompt_options

['Below is a math problem you are to solve (positive numerical answer):\n"{}"\nTo accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!\nWrite the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.\n\nApproach:',
 'Below is a math problem you are to solve (positive numerical answer!):\n"{}"\nAnalyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n']

In [ ]:
template = "Hello, {}!"
a="d"
result = template.format(a)
print(result)  # 出力: Hello, World!

Hello, d!


In [ ]:
# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# 解答を評価する関数
def evaluate_answer(answer):
    try:
        return int(answer.strip())  # ダミーの評価: 数値に変換できれば有効とする
    except:
        return None

# プロンプトを生成する関数
def generate_prompt(problem, past_outputs=None):
    counts = np.array([2, 3])
    draw = choice(prompt_options, 1, p=counts/counts.sum())
    print(draw[0].format(problem))
    if past_outputs:
        previous_output = "\n".join(past_outputs)
        prompt = f"User: {draw[0].format(problem)}\n{previous_output}"
    else:
        prompt = f"User: {draw[0].format(problem)}"
    return prompt

# メインの関数
def main():
    problem = input("Enter a math problem: ")
    iterations = int(input("Number of iterations (1-100): "))
    total_time_limit = int(input("Total time limit (seconds, 10-3600): "))
    single_attempt_time_limit = int(input("Single attempt time limit (seconds, 1-300): "))
    print(problem)

    start_time = time.time()
    final_answer = None
    past_outputs = []
    answers = []
    print(1)
    initial_prompt = generate_prompt(problem)
    print(2)
    prompt = initial_prompt

    for attempt in range(iterations):
        if time.time() - start_time > total_time_limit:
            print("Total time limit exceeded.")
            break

        attempt_start_time = time.time()
        while time.time() - attempt_start_time < single_attempt_time_limit:
            model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
            generation_output = model.generate(**model_inputs,
                                               max_new_tokens=2048,
                                               do_sample=True,
                                               temperature=0.9,
                                               top_p=1.0,
                                               num_return_sequences=1)

            generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
            print(f"Attempt {attempt + 1} Output:")
            print(generated_text)

            past_outputs.append(generated_text)

            answer = evaluate_answer(generated_text)
            if answer is not None:
                answers.append(answer)
                break  # 有効な解答が見つかった場合、内側のループを抜ける

            time.sleep(1)  # 処理時間をシミュレートするための小さな遅延

        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"Final Answer: {final_answer}")
            break
    else:
        if answers:
            final_answer = Counter(answers).most_common(1)[0][0]
            print(f"Final Answer: {final_answer}")
        else:
            print("Could not find a valid answer within the specified attempts.")
            if past_outputs:
                valid_answers = [evaluate_answer(output) for output in past_outputs if evaluate_answer(output) is not None]
                if valid_answers:
                    final_answer = choice(valid_answers, 1)[0]
                    print(f"Randomly selected answer from past attempts: {final_answer}")

if __name__ == "__main__":
    main()

Enter a math problem: "あああ"
Number of iterations (1-100): 3
Total time limit (seconds, 10-3600): 100
Single attempt time limit (seconds, 1-300): 10
"あああ"
1


IndexError: Replacement index 1 out of range for positional args tuple

##アプリの起動

In [36]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://335d-35-204-227-67.ngrok-free.app" -> "http://localhost:8501"


In [37]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100% 2/2 [00:03<00:00,  1.50s/it]
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100% 2/2 [00:02<00:00,  1.37s/it]
Setting `pad_token_id` to `eos_

In [ ]:
# import subprocess
# from pyngrok import ngrok

# # ngrokの認証トークンを設定します
# NGROK_AUTH_TOKEN = "2dTzmleYxq1mTfiDFbkxE1wvHG3_2cw3k3t36UDyDFUr29TUz"
# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Streamlitアプリをサブプロセスとして実行します
# streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# # ngrokトンネルを作成して、ポート8501を指定します
# public_url = ngrok.connect(8501)
# print(f'Streamlit app is live at: {public_url}')

Streamlit app is live at: NgrokTunnel: "https://5b61-34-41-54-144.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# streamlit_process.terminate()